# Assignment 3 - Exams Schedule Generator Using Genetic Algorithm

## Overview
This Assignment implements a Genetic Algorithm (GA) to generate an optimal examination schedule for a university. The algorithm ensures that scheduling constraints are met while optimizing certain preferences.

## Features
- Reads input data from provided Excel files.
- Implements a Genetic Algorithm from scratch using only `numpy` and `pandas`.
- Uses **tournament_selection** for chromosome selection.
- Supports **One point** crossover technique.
- Applies penalties for constraint violations and rewards for optimization criteria.

## Fitness Function
The fitness function evaluates schedules based on:
### **Hard Constraints (Must be Satisfied)**
- Every course must have an exam scheduled.
- No student should have overlapping exams.
- Exams are only scheduled from Monday to Friday, 9 AM - 5 PM.
- Each exam must have an invigilating teacher.
- No teacher is assigned to multiple exams at the same time.
- No teacher is assigned consecutive exam invigilation duties.

### **Soft Constraints (Optimization Criteria)**
- A common break on Friday from 1–2 PM.
- Avoiding back-to-back exams for students.
- Preferably scheduling Management (MG) exams before Computer Science (CS) exams.
- Two-hour break in a week for faculty meetings.


## Output Format
The final output should:
- Show The best Fitness accoridng to the Generation.
- Display a final valid schedule satisfying all hard constraints and at least 3 soft constraints.

## Running the Code
Ensure you have the required dependencies installed:
```bash
pip install numpy pandas
```
Run the notebook or execute the script to generate the exam schedule.



In [1]:
import pandas as pd
import numpy as np
import random

# Data Loading and Preprocessing

## Understanding the Data
- The script reads four CSV files: `courses.csv`, `studentCourse.csv`, `studentNames.csv`, and `teachers.csv`.
- It extracts the list of courses and teachers from their respective datasets.
- A dictionary is created to map each student to their enrolled courses.
- The `Course Code` field in `studentCourse.csv` contains multiple courses separated by commas, which are split and stored properly.
- This mapping is essential for ensuring that students are not scheduled for overlapping exams.

In [2]:
courses_df = pd.read_csv("courses.csv")
student_course_df = pd.read_csv("studentCourse.csv")
student_names_df = pd.read_csv("studentNames.csv")
teachers_df = pd.read_csv("teachers.csv")

total_courses = courses_df["Course Code"].tolist()
teachers = teachers_df["Names"].tolist()


student_course_dict = {}

for _, row in student_course_df.iterrows():
    student_name = row["Student Name"]
    course_codes = row["Course Code"].split(",") 

    if student_name in student_course_dict:
        student_course_dict[student_name].extend(course_codes)
    else:
        student_course_dict[student_name] = course_codes


# Population Initialization

## Understanding Population Initialization
- A population of potential schedules is generated based on a given population size.
- Each schedule consists of multiple exams, where each exam is assigned:
  - A random weekday (Monday to Friday).
  - A random start time between 9 AM and 5 PM.
  - A randomly chosen teacher.
- Constraints are enforced to avoid scheduling conflicts:
  - No two exams share the same time slot.
  - A teacher cannot be assigned multiple exams at the same time.
- The generated population serves as the initial set of solutions for the Genetic Algorithm.

In [3]:
def initialize_population(pop_size, courses, teachers):
    population = []
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    
    for _ in range(pop_size):
        schedule = []
        assigned_slots = set()  
        teacher_schedule = {}  
        
        for course in courses:
            while True:
                day = np.random.choice(weekdays)
                start_time = np.random.randint(9, 17)
                teacher = np.random.choice(teachers)

                if (day, start_time) not in assigned_slots and (teacher, day, start_time) not in teacher_schedule:
                    assigned_slots.add((day, start_time))
                    teacher_schedule[(teacher, day, start_time)] = True
                    
                    exam = {
                        "course_code": course,
                        "day": day,
                        "start_time": start_time,
                        "teacher": teacher
                    }
                    schedule.append(exam)
                    break  

        population.append(schedule)
    
    return population


# Fitness Evaluation

## Understanding the Fitness Function
- The fitness function assesses the quality of a generated exam schedule.
- It applies **penalties** for violations of hard constraints:
  - Overlapping exams for students.
  - Teachers assigned multiple exams at the same time.
  - Exams scheduled outside valid weekdays or time slots.
  - Consecutive invigilation duties for teachers.
  - Students having back-to-back exams.
  - Missing exams for required courses.
- It applies **rewards** for optimization criteria:
  - A common Friday break from 1–2 PM for all students and teachers.
- The final fitness score is calculated as:
  
  **Fitness Score = Rewards - Penalties**
  
- A list of violated constraints is also returned to help refine the schedule.
- This ensures that the algorithm iteratively improves schedules while maintaining feasibility.

In [4]:
def fitness(schedule, student_course_dict):
    penalty = 0
    reward = 0
    violated_constraints = []

    # Checking for student exam clashes
    for student, courses in student_course_dict.items():
        student_exams = [exam for exam in schedule if exam["course_code"] in courses]
        times = [(exam["day"], exam["start_time"]) for exam in student_exams]

        if len(times) != len(set(times)):  # Overlapping exams
            penalty += 50
            violated_constraints.append(f"Student {student} has overlapping exams.")

    # Checking for teacher clashes
    teacher_schedule = {}
    for exam in schedule:
        teacher = exam["teacher"]
        time_slot = (exam["day"], exam["start_time"])

        if teacher in teacher_schedule:
            if time_slot in teacher_schedule[teacher]:  
                penalty += 20
                violated_constraints.append(f"Teacher {teacher} has overlapping exams.")
            else:
                teacher_schedule[teacher].add(time_slot)
        else:
            teacher_schedule[teacher] = {time_slot}

    # Checking for valid weekdays
    for exam in schedule:
        if exam["day"] not in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]:
            penalty += 20
            violated_constraints.append(f"Exam for {exam['course_code']} scheduled on invalid day: {exam['day']}.")

    # Checking for valid time range
    for exam in schedule:
        if not (9 <= exam["start_time"] < 17):
            penalty += 20
            violated_constraints.append(f"Exam for {exam['course_code']} scheduled outside working hours: {exam['start_time']}.")

    # Rewarding Friday common break from 1-2 PM
    for exam in schedule:
        if exam["day"] == "Friday" and exam["start_time"] == 13:
            reward += 80

    # Checking for consecutive exams for students
    for student, courses in student_course_dict.items():
        student_exams = [e for e in schedule if e["course_code"] in courses]
        student_times = sorted([(e["day"], e["start_time"]) for e in student_exams])

        for i in range(len(student_times) - 1):
            day1, time1 = student_times[i]
            day2, time2 = student_times[i + 1]

            if day1 == day2 and time2 == time1 + 1: 
                penalty += 20
                violated_constraints.append(f"Student {student} has back-to-back exams on {day1}.")

    # Checking if every course has an exam scheduled
    scheduled_courses = {exam["course_code"] for exam in schedule}
    missing_courses = [course for course in student_course_dict.keys() if course not in scheduled_courses]
    if missing_courses:
        penalty += 15 * len(missing_courses)
        violated_constraints.append(f"Missing exams for courses: {', '.join(missing_courses)}.")

    # Checking for back-to-back teacher invigilation
    for teacher, slots in teacher_schedule.items():
        sorted_slots = sorted(slots)
        for i in range(len(sorted_slots) - 1):
            day1, time1 = sorted_slots[i]
            day2, time2 = sorted_slots[i + 1]
            if day1 == day2 and time2 == time1 + 1:
                penalty += 15
                violated_constraints.append(f"Teacher {teacher} has consecutive invigilation duties on {day1}.")

    # Returning both fitness score and violated constraints
    return reward - penalty, violated_constraints


# Selection Process

## Understanding Tournament Selection
- Tournament selection is used to choose the best candidates from the population.
- A subset of individuals (tournament size) is randomly selected.
- The individual with the highest fitness score in this subset is chosen as the winner.
- This method ensures that stronger candidates are more likely to be selected while still allowing diversity in selection.

In [5]:
def tournament_selection(population, fitnesses, tournament_size=5):
    selected = np.random.choice(len(population), tournament_size, replace=False)
    best_index = max(selected, key=lambda i: fitnesses[i])
    return population[best_index]

# Crossover Process

## Understanding Crossover
- Crossover is used to combine genetic information from two parent schedules to create new offspring.
- A random crossover point is chosen within the schedule.
- The first part of each child is taken from one parent, and the second part from the other.
- The algorithm ensures that duplicate exams are avoided in the offspring by checking for conflicts.
- This operation introduces genetic diversity and helps refine better schedules over generations.

In [6]:
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1) - 1)
    
    child1 = []
    child2 = []

    for i in range(len(parent1)):
        if i < crossover_point:
            child1.append(parent1[i])
            child2.append(parent2[i])
        else:
            exam_from_parent2 = parent2[i]
            exam_from_parent1 = parent1[i]

            child1.append(exam_from_parent2 if exam_from_parent2 not in child1 else parent1[i])
            child2.append(exam_from_parent1 if exam_from_parent1 not in child2 else parent2[i])

    return child1, child2


# Mutation Process

## Understanding Mutation
- Mutation introduces small random changes to an individual schedule to maintain genetic diversity.
- Each exam in the schedule has a small probability (mutation rate) of being altered.
- Changes include:
  - Assigning the exam to a new day and time.
  - Reassigning a different invigilating teacher.
- Constraints are enforced to prevent conflicts:
  - No two exams should share the same time slot.
  - No teacher should be assigned to multiple exams simultaneously.
- This process helps prevent premature convergence and ensures diverse solutions.

In [7]:
def mutate(schedule, mutation_rate, teachers):
    for exam in schedule:
        if np.random.rand() < mutation_rate:
            new_day = np.random.choice(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])
            new_time = np.random.randint(9, 17)
            new_teacher = np.random.choice(teachers)

            if not any(e["start_time"] == new_time and e["day"] == new_day for e in schedule):
                exam["day"] = new_day
                exam["start_time"] = new_time

            if not any(e["teacher"] == new_teacher and e["start_time"] == new_time and e["day"] == new_day for e in schedule):
                exam["teacher"] = new_teacher

    return schedule

# Genetic Algorithm Execution

## Understanding the Genetic Algorithm Process
- The algorithm starts by initializing a population of random schedules.
- It runs for a predefined number of generations to evolve better schedules.
- Each generation follows these steps:
  1. **Fitness Evaluation:** Each schedule is scored based on constraint fulfillment.
  2. **Elitism:** The top-performing schedules are preserved for the next generation.
  3. **Selection:** Tournament selection is used to choose parents for reproduction.
  4. **Crossover & Mutation:** New schedules are generated through crossover and mutation.
  5. **Replacement:** The new population replaces the old one, maintaining diversity.
- The best schedule found across all generations is returned as the final solution.


In [14]:
def genetic_algorithm(courses, teachers, student_course_data, pop_size=50, generations=100, mutation_rate=0.05):
    population = initialize_population(pop_size, courses, teachers)
    best_overall = []
    best_overall_fitness = -float('inf')  
    
    elite_count = max(3, int(0.1 * pop_size))

    for gen in range(generations):
        
        fitness_results = [fitness(individual, student_course_data) for individual in population]
        fitnesses = [f[0] for f in fitness_results]  # Extract fitness scores
        violations = [f[1] for f in fitness_results]  # Extract violated constraints
        
        sorted_indices = np.argsort(fitnesses)[::-1] 
        sorted_population = [population[i] for i in sorted_indices]
        sorted_fitnesses = [fitnesses[i] for i in sorted_indices]
        sorted_violations = [violations[i] for i in sorted_indices]
        
        new_population = sorted_population[:elite_count]
        
        if sorted_fitnesses[0] > best_overall_fitness:
            best_overall_fitness = sorted_fitnesses[0]
            best_overall = sorted_population[0][:]
            best_overall_violations = sorted_violations[0]
        
        print(f"Generation {gen}: Best Fitness = {sorted_fitnesses[0]}")
        if best_overall_violations:
            print("Violated Constraints:", best_overall_violations)
        else:
            print("All constraints satisfied!")
        # print(len(new_population))
        # print(new_population)
        for i in range(pop_size - elite_count):
            parent1 = tournament_selection(sorted_population, sorted_fitnesses)
            parent2 = tournament_selection(sorted_population, sorted_fitnesses)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate, teachers)
            child2 = mutate(child2, mutation_rate, teachers)
            new_population.extend([child1, child2])

        population = new_population[:pop_size]
        # print([fitness(individual, student_course_data) for individual in population])

    return best_overall, best_overall_fitness,best_overall_violations

# Displaying the Final Schedule

## Understanding Schedule Display
- The final schedule is presented in a structured format using a DataFrame.
- The schedule is sorted by day and start time for better readability.
- The fitness score of the schedule is displayed alongside the table.
- This helps visualize the effectiveness of the evolved schedule and ensures constraints are met.

In [12]:
def display_schedule(schedule,fitness,violation):
    df = pd.DataFrame(schedule)
    df = df.sort_values(by=["day", "start_time"])
    print(df)
    print(fitness)
    print("Violation : ",violation)


In [15]:
best_schedule,best_finess,bestviolation = genetic_algorithm(total_courses, teachers, student_course_dict, pop_size=50, generations=5, mutation_rate=0.05)

display_schedule(best_schedule,best_finess,bestviolation)

Generation 0: Best Fitness = -2560
Violated Constraints: ['Student Sarah N Md Sallehuddin Khan has back-to-back exams on Friday.', 'Student Nabila Altaf has back-to-back exams on Tuesday.', 'Student Nosheen Maqbool has back-to-back exams on Friday.', 'Student Nima Dirie has back-to-back exams on Tuesday.', 'Student Tina Nasersharif has back-to-back exams on Friday.', 'Student Iram Meharban has back-to-back exams on Friday.', 'Missing exams for courses: Sam D Edwards, Sheila Hughton, Yasmin Ahmed, Sarah N Md Sallehuddin Khan, Sarah Nolasco, Jenna Riley, Usman Rafiq, Reem N Hassan, Sarah Hinett, Kamal Anwar, Mika Tatsumoto, Muhammad Ijaz-Ul-Haq, Abdul Gafur, Ana Vukojevic, Arooba Zahoor, Ahmad F Yang Abd Talib, Natasha Leeson, Ramesh R Singh, Sara Zamberlan, Adam N Starling, Maria M Ponce Carpio, Iram Matloob, Sarah J Roberts, Maria Lytras, Mohammad Abir, Nabila Altaf, Maria A Grenfell, Mohamed A Baalousha, Ayan Lowe, Mohammed I Al Arfaj, Mohammed Riyaz Shahul Hameed Mohammed Yakub, Sara

# Saving the Schedule to a File

## Understanding Schedule Saving
- The generated exam schedule is saved to a text file for record-keeping.
- The schedule is sorted by day and start time before being written to the file.
- A structured format is used, including headers and separators.
- The fitness score of the schedule is also included in the file.
- This ensures that the final schedule can be easily reviewed and shared.

In [16]:
def save_schedule_to_file(schedule, fitness,violation, filename="exam_schedule.txt"):

    df = pd.DataFrame(schedule)
    df = df.sort_values(by=["day", "start_time"]) 

    # Write to a text file
    with open(filename, "w") as file:
        file.write("Exam Schedule\n")
        file.write("="*50 + "\n")
        df_string = df.to_string(index=False)  
        file.write(df_string + "\n")
        file.write("="*50 + "\n")
        file.write(f"Fitness Score: {fitness}\n")
        file.write(f"Violations : {violation}\n")

    print(f"Schedule saved to {filename}")


save_schedule_to_file(best_schedule, best_finess,bestviolation, "final_schedule.txt")


Schedule saved to final_schedule.txt
